In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la passe,45.5549,-0.8967,18.90,79,1,3.78,FR,2024-05-11
1,1,hermanus,-34.4187,19.2345,12.11,67,55,2.16,ZA,2024-05-11
2,2,kununurra,-15.7667,128.7333,31.99,43,40,3.60,AU,2024-05-11
3,3,stanley,54.8680,-1.6985,15.01,82,0,0.51,GB,2024-05-11
4,4,udachny,66.4167,112.4000,-3.64,62,100,1.82,RU,2024-05-11
...,...,...,...,...,...,...,...,...,...,...
617,617,sumbawanga,-7.9667,31.6167,21.89,68,40,3.09,TZ,2024-05-11
618,618,urucara,-2.5364,-57.7600,23.28,98,78,0.87,BR,2024-05-11
619,619,deqen,29.9618,90.7188,9.78,16,100,2.94,CN,2024-05-11
620,620,jaicos,-7.3592,-41.1378,18.39,72,100,0.67,BR,2024-05-11


#### STEP ONE

In [3]:
# Configure the map plot_1
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"
)

# Display the map plot_1
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

In [4]:

northern_hemi = city_data_df[city_data_df['Lat'] >= 0]
southern_hemi = city_data_df[city_data_df['Lat'] < 0]

if not northern_hemi.empty:
    idealwthr_north = northern_hemi[
    (northern_hemi["Humidity"] <= 50) &
    (northern_hemi["Max Temp"] <= 40) &
    (northern_hemi["Max Temp"] >= 20) &
    (northern_hemi["Cloudiness"] <= 70) &
    (northern_hemi["Wind Speed"] >= 5)
    ]
else:
    idealwthr_north = northern_hemi

if not southern_hemi.empty:
    idealwthr_south = southern_hemi[
    (southern_hemi["Humidity"] <= 70) &
    (southern_hemi["Max Temp"] <= 15) &
    (southern_hemi["Max Temp"] >= -5) &
    (southern_hemi["Cloudiness"] <= 60) &
    (southern_hemi["Wind Speed"] <= 6)
    ]
else:
    idealwthr_south = southern_hemi

idealwthr = pd.concat([idealwthr_north, idealwthr_south])
idealwthr = idealwthr.dropna()
print(f"The amount of ideal cities I can visit is: {len(idealwthr)}.")
idealwthr

The amount of ideal cities I can visit is: 19.


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
131,131,hohhot,40.8106,111.6522,20.16,10,0,9.00,CN,2024-05-11
155,155,tamanrasset,22.7850,5.5228,25.95,12,0,8.23,DZ,2024-05-11
160,160,afif,23.9065,42.9172,32.13,29,5,8.72,SA,2024-05-11
319,319,kapoeta,4.7717,33.5903,31.68,40,25,8.98,SS,2024-05-11
373,373,xinzhou,38.4092,112.7333,23.67,10,0,9.09,CN,2024-05-11
395,395,yulinshi,38.2906,109.7494,23.79,11,0,7.39,CN,2024-05-11
429,429,jaisalmer,26.9147,70.9181,36.90,27,0,5.14,IN,2024-05-11
487,487,brahmapur,19.3167,84.7833,38.15,39,0,5.81,IN,2024-05-11
492,492,laojunmiao,39.8333,97.7333,22.25,9,0,5.08,CN,2024-05-11
539,539,riyadh,24.6877,46.7219,34.08,13,10,5.15,SA,2024-05-11


In [5]:
hotel_order = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = idealwthr[hotel_order]
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\AndBe\AppData\Local\Temp\ipykernel_5140\3216582088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
131,hohhot,CN,40.8106,111.6522,10,
155,tamanrasset,DZ,22.7850,5.5228,12,
160,afif,SA,23.9065,42.9172,29,
319,kapoeta,SS,4.7717,33.5903,40,
373,xinzhou,CN,38.4092,112.7333,10,
395,yulinshi,CN,38.2906,109.7494,11,
429,jaisalmer,IN,26.9147,70.9181,27,
487,brahmapur,IN,19.3167,84.7833,39,
492,laojunmiao,CN,39.8333,97.7333,9,
539,riyadh,SA,24.6877,46.7219,13,


In [6]:
radius = 10000
base_url = "https://api.geoapify.com/v2/places"

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    lat, lng = row['Lat'], row['Lng']
    categories = "accommodation.hotel"
    filters = f"circle:{lng},{lat},{radius}"
    bias = f"proximity:{lng},{lat}"

    params = {
        "categories": categories,
        "filter": filters,
        "bias": bias,
        "apiKey": geoapify_key
    }

    response = requests.get(base_url, params=params)
    name_address = response.json() 

    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "No hotel found"

    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hohhot - nearest hotel: Yue Jia Budget Hotel (foreigners approved)
tamanrasset - nearest hotel: فندق الأمان
afif - nearest hotel: Samaya Royal Afif
kapoeta - nearest hotel: Kuleu
xinzhou - nearest hotel: No hotel found
yulinshi - nearest hotel: 榆溪大酒店
jaisalmer - nearest hotel: Jeet Mahal
brahmapur - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
riyadh - nearest hotel: White Palace Hotel
murun-kuren - nearest hotel: No hotel found
hancheng - nearest hotel: No hotel found
renqiu - nearest hotel: 华北油田宾馆
sabha - nearest hotel: فندق الكاظم-سبها
hermanus - nearest hotel: Aloe guest house
bredasdorp - nearest hotel: Victoria Hotel
queenstown - nearest hotel: Queens Hotel
coquimbo - nearest hotel: Hotel Iberia
oudtshoorn - nearest hotel: The Pearl


,City,Country,Lat,Lng,Humidity,Hotel Name
131,hohhot,CN,40.8106,111.6522,10,Yue Jia Budget Hotel (foreigners approved)
155,tamanrasset,DZ,22.7850,5.5228,12,فندق الأمان
160,afif,SA,23.9065,42.9172,29,Samaya Royal Afif
319,kapoeta,SS,4.7717,33.5903,40,Kuleu
373,xinzhou,CN,38.4092,112.7333,10,No hotel found
395,yulinshi,CN,38.2906,109.7494,11,榆溪大酒店
429,jaisalmer,IN,26.9147,70.9181,27,Jeet Mahal
487,brahmapur,IN,19.3167,84.7833,39,No hotel found
492,laojunmiao,CN,39.8333,97.7333,9,No hotel found
539,riyadh,SA,24.6877,46.7219,13,White Palace Hotel


In [8]:
city_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat',
    geo=True, 
    tiles='OSM', 
    frame_width=700,
    frame_height=500,
    hover_cols=['City', 'Hotel Name', 'Country'], 
    title="Ideal Cities",
    size= 50,
    color = "Country"
)

city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,City,Hotel Name)